In [1]:
from tqdm import tqdm_notebook as tqdm

# Cross Validation

In [ ]:
import os
import sys
sys.path.append('../models/topic_model/')
import sampler_cy
import json

## data
data_input = []
with open('../models/topic_model/labeled_data/doc_vecs.json') as data:
    for line in data:
        data_input.append(json.loads(line))
vocab = open('../models/topic_model/labeled_data/vocab.txt').read().split('\n')[:-1]

from sklearn.model_selection import KFold
import numpy as np 
kf = KFold(n_splits=5)

labeled_data_idx = list(filter(lambda x: x[1]['has_labels'], list(enumerate(data_input))))
labeled_data_idx = np.array(list(map(lambda x: x[0], labeled_data_idx)))

num_iterations = 1000
kfold = 0

for train_idx, test_idx in kf.split(labeled_data_idx):
    outdir = os.path.join('crossfold-topic-model', 'fold-%d' % kfold)
    os.makedirs(outdir, exist_ok=True)
    ## 
    test_examples = labeled_data_idx[test_idx]
    for idx in test_examples:
        data_input[idx]['has_labels'] = False

    ## sampler
    sampler = sampler_cy.BOW_Paragraph_GibbsSampler(data_input, vocab=vocab)
    sampler.initialize()
    for i in tqdm(range(num_iterations)):
        sampler.sample_pass()
    sampler.save_state(outdir)
    np.savetxt(os.path.join(outdir, 'test-examples.txt'), test_examples)    
    
    kfold += 1
    del sampler

# Read in Data

In [12]:
import numpy as np 
import os
import json
import pandas as pd 

In [24]:
label_mapper_inverse = {
    0: 'LEAD',
    1: 'CIRC',
    2: 'MAIN',
    3: 'VERB',
    4: 'CONS',
    5: 'PREV',
    6: 'EVAL',
    7: 'HIST',
    8: 'EXPE',
    9: 'SECD',
    10: 'EXPL',
    -1: np.nan
}

In [13]:
os.listdir('../models/topic_model/labeled_output/model-state-iter-9999.pkl/')

['doc_by_wordtopic.txt',
 'pardoc_to_type.txt',
 'partype_by_wordtopic.txt',
 'switching_variable_counts.txt',
 'vocab_by_wordtopic.txt']

In [14]:
pardoc_to_type = json.load(open('../models/topic_model/labeled_output/model-state-iter-999.pkl/pardoc_to_type.txt'))

In [16]:
pardoc_to_type_df = pd.DataFrame(pardoc_to_type)

In [30]:
(pardoc_to_type_df.unstack().dropna()
 .value_counts()
 .reset_index()
 .assign(index=lambda df: df['index'].map(label_mapper_inverse))
 .set_index('index')[0]
)

index
VERB    22693
CIRC    11951
MAIN     8508
PREV     6549
EVAL     5784
CONS     5672
HIST     4020
LEAD     3426
EXPE     2493
SECD     1308
EXPL      224
NaN         3
Name: 0, dtype: int64